In [ ]:
import pandas as pd
import scipy as sp
import numpy as np
import array
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import gensim
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
cd /content/drive/MyDrive/amazon_contest/fake_news

/content/drive/MyDrive/amazon_contest/fake_news


In [ ]:
train_data = pd.read_excel("Training_fake_news.xlsx")
valid_data = pd.read_excel("Validation_Fake_news.xlsx")
test_data = pd.read_excel("Test_Fake_news.xlsx")
# print(training_data.head())
# print(valid_data.head())
# print(test_data.head())

train_label = train_data["label"]
print(type(train_label))
# train_label.head()
#print(train_data['label'].value_counts())
train_data.drop(columns=["label"],inplace=True)
print(type(train_data))
#training_data.head()

valid_label = valid_data["label"]
valid_data.drop(columns=["label"],inplace=True)

test_label = test_data["label"]
test_data.drop(columns=["label"],inplace=True)

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
def pre_processing(tweets):

  #removing punctuations
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~0123456789'''
  new_tweets = []
  for sentences in tweets:
    x = ""
    for alphabets in sentences:
      if alphabets not in punc:
        x = x + alphabets
    new_tweets.append(x)
  #print(new_train_tweets)


  # doing stemming on sentecnes
  tweet_stemming = []
  ps = PorterStemmer()
  for sentences in new_tweets:
    s1 = []
    temp = word_tokenize(sentences) #converts sentence to words
    for words in temp:
      t = ps.stem(words)
      s1.append(t)
    tweet_stemming.append(s1)

  # print(tweet_stemming[0])
  # converting sentences to list of words
  # train_tweet_words = []
  # for sentences in tweet_stemming:
  #   temp = sentences.split()
  #   train_tweet_words.append(temp)

  #removing stop words
  new_train_tweets = []
  stop_words = set(stopwords.words('english'))
  for sentences in tweet_stemming:
    temp = []
    for words in sentences:
      if words not in stop_words:
        temp.append(words)
    new_train_tweets.append(temp)

  #converting list of words to sentences
  all_sentences = []
  for lists in new_train_tweets:
    x = ' '.join(lists)
    all_sentences.append(x)
  
  return all_sentences

In [ ]:
train_tweets = list(train_data["tweet"])
test_tweets = list(test_data["tweet"])
valid_tweets = list(valid_data["tweet"])

#coverting all to lower
for i in range(len(train_tweets)):
  train_tweets[i] = train_tweets[i].lower()

for i in range(len(valid_tweets)):
  valid_tweets[i] = valid_tweets[i].lower()

for i in range(len(test_tweets)):
  test_tweets[i] = test_tweets[i].lower()

train_tweets = pre_processing(train_tweets)
test_tweets = pre_processing(test_tweets)
valid_tweets = pre_processing(valid_tweets)

#creating data_frame for training
df_train = pd.DataFrame(train_tweets)
df_train.columns = ["tweet"]
df_train["label"] = train_label
df_train['label'] = df_train['label'].map({'real': 1, 'fake': 0})
print(df_train.head(10))

#creating data_frame for validation
df_valid = pd.DataFrame(valid_tweets)
df_valid.columns = ["tweet"]
df_valid["label"] = valid_label
df_valid['label'] = df_valid['label'].map({'real': 1, 'fake': 0})
print(df_valid.head(10))

#creating data_frame for test
df_test = pd.DataFrame(test_tweets)
df_test.columns = ["tweet"]
df_test["label"] = test_label
df_test['label'] = df_test['label'].map({'real': 1, 'fake': 0})
print(df_test.head(10))


                                               tweet  label
0  cdc current report death gener discrep death c...      1
1  state report death small rise last tuesday sou...      1
2  polit correct woman almost use pandem excus re...      0
3  indiafightscorona covid test laboratori india ...      1
4  popul state gener larg case count look new cas...      1
5  covid act found averag person illinoi covid in...      1
6  test posit covid symptom stay home away peopl ...      1
7  obama call trump ’ coronaviru respons chaotic ...      0
8  clearli obama administr leav ani kind game pla...      0
9  retraction—hydroxychloroquin chloroquin withou...      0
                                               tweet  label
0  chines convert islam realis muslim wa affect c...      0
1  peopl diamond princess cruis ship intial test ...      0
2            covid caus bacterium viru treat aspirin      0
3  mike penc rnc speech prais donald trump ’ covi...      0
4  ski edconwayski explain latest covid 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
vocab_size = 5000

In [ ]:
onehot_repr = [one_hot(words,vocab_size) for words in df_train['tweet']]
print(onehot_repr)

[[892, 616, 95, 4863, 1486, 2744, 4863, 4200, 3128, 4772, 1553, 81, 4863, 2555, 2652, 3455, 3486, 305], [3438, 95, 4863, 1553, 3286, 3769, 4319, 2220, 3438, 95, 4863, 2418], [1193, 1118, 1547, 4759, 1021, 4969, 925, 4498, 1753, 1412, 1261, 1272, 4744], [926, 451, 3012, 3490, 2105, 4563, 708, 3012, 3773, 2913, 3946, 2718, 2295, 4599, 2325], [402, 3438, 1486, 2396, 3338, 4200, 4052, 2093, 3338, 139, 2036, 305, 3381, 3438, 3736, 3338, 139, 2036, 4921, 3433, 1957, 4671, 216, 3206, 1533, 4629, 4061, 385, 3304, 4979], [451, 1808, 1972, 4951, 867, 454, 451, 549, 3486, 4065, 3736, 549, 3734, 3198, 1332, 2713, 384, 4433, 5, 1982, 3859, 919, 600, 4966, 723], [3012, 1417, 451, 1639, 166, 3774, 193, 3486, 986, 892, 570, 887, 1780, 451, 549, 3158, 2553], [4689, 710, 1757, 570, 1272, 2712, 2220, 4729, 1899], [3628, 4689, 2502, 2083, 4226, 4237, 2627, 4278, 1953, 1619, 4058], [2147, 2671, 1805, 1260, 4575, 451, 202, 3350, 1512, 2174, 1388], [917, 3067, 2031, 813, 666, 3022, 321, 3260, 3069, 1619, 451

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0  892 ... 3455 3486  305]
 [   0    0    0 ...   95 4863 2418]
 [   0    0    0 ... 1261 1272 4744]
 ...
 [   0    0    0 ... 1966 3371  126]
 [   0    0    0 ... 4365 1396 4016]
 [3769 3338  451 ... 3111 4864  822]]


In [ ]:
embedding_vec_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vec_features,input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X_final = np.array(embedded_docs)
y_final = np.array(df_train['label'])

In [ ]:
X_final.shape,y_final.shape

((6420, 20), (6420,))

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final, test_size = 0.33, random_state = 42)

In [ ]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 10, batch_size = 64)

Epoch 1/10
68/68 [==============================] - 5s 42ms/step - loss: 0.4898 - accuracy: 0.7712 - val_loss: 0.3269 - val_accuracy: 0.8532
Epoch 2/10
68/68 [==============================] - 2s 32ms/step - loss: 0.2167 - accuracy: 0.9165 - val_loss: 0.2584 - val_accuracy: 0.8919
Epoch 3/10
68/68 [==============================] - 2s 31ms/step - loss: 0.1129 - accuracy: 0.9581 - val_loss: 0.2808 - val_accuracy: 0.8844
Epoch 4/10
68/68 [==============================] - 2s 31ms/step - loss: 0.0781 - accuracy: 0.9767 - val_loss: 0.4405 - val_accuracy: 0.8849
Epoch 5/10
68/68 [==============================] - 2s 31ms/step - loss: 0.0438 - accuracy: 0.9877 - val_loss: 0.4537 - val_accuracy: 0.8853
Epoch 6/10
68/68 [==============================] - 2s 31ms/step - loss: 0.0298 - accuracy: 0.9921 - val_loss: 0.6619 - val_accuracy: 0.8721
Epoch 7/10
68/68 [==============================] - 2s 32ms/step - loss: 0.0373 - accuracy: 0.9886 - val_loss: 0.4921 - val_accuracy: 0.8764
Epoch 8/10
68

In [ ]:
y_pred = model.predict(X_test)
for i in range(len(y_pred)):
  if(y_pred[i] <= 0.5):
    y_pred[i] = 0
  else:
    y_pred[i] = 1


print('\n Accuracy: ', accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred))


 Accuracy:  0.8773006134969326
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      1004
           1       0.88      0.89      0.88      1115

    accuracy                           0.88      2119
   macro avg       0.88      0.88      0.88      2119
weighted avg       0.88      0.88      0.88      2119



In [ ]:
_